# The frequency-Hough transform tutorial
### Searching for continuous gravitational waves from isolated, asymmetricly rotating neutron stars
Andrew L. Miller

National Institute for Subatomic Physics / Utrecht University

#### Searches for continuous waves from isolated, deformed neutron stars are broadly classified into three forms, depending on how much we know about the source: targeted (known position, rotational frequency and spin-down), directed (known position, unknown frequency/spin-down) and all-sky (all parameters unknown).

#### There are benefits and drawbacks to each kind of search -- see arXiv:2206.06447 for a recent review.

###### This tutorial will focus on a particular algorithm for all-sky searches for continuous gravitational waves: the frequency-Hough transform. This method maps points in the time/frequency plane of the detector to lines in the frequency/spin-down plane of the source. See arXiv:1407.8333 for more details.

#### Note: The frequency-Hough transform algorithm and its implementation for CW searches has been designed and implemented for real searches by the Virgo Rome group of the LVK collaboration

# Import stuff

In [1]:
# this will take a few minutes on first run, then should use cached files and be fast
!pip install pyhough
!pip install pyfstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 2.6 MB/s eta 0:00:0031m2.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 8.2 MB/s eta 0:00:00


In [2]:
#----------------------
# Setting up the environment
#----------------------
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/2024_OpenDataWorkshop_cw
!ls

ModuleNotFoundError: No module named 'google'

In [ ]:
import os
import pyfstat
import lal
lal.swig_redirect_standard_output_error(False)
import numpy as np
import pyhough
from pyhough import pm
from pyhough import hm
#### for some reason pm and hm need to be imported separately
# make sure to put these after the pyfstat import, to not break notebook inline plots
import matplotlib.pyplot as plt
%matplotlib inline


# Simulate a CW signal and compute a spectrogram

Compute the spectrogram of a set of SFTs. This is useful to produce
visualizations of the Doppler modulation of a CW signal.


In [ ]:
# general setup
label = "PyhoughExample"
outdir = os.path.join("Mockdata", label)
logger = pyfstat.set_up_logger(label=label, outdir=outdir)

In [ ]:
# properties of the GW data
sftfilepath = "Mockdata/H-7008_H1_1800SFT_SingleDetectorGaussianNoiseSignalInjected-1238166018-12614400.sft"
tstart = 1238166018
duration = 0.4 * 365 * 86400
detectors = "H1"
Tsft = 1800
depth = 5
sqrtSX = 1.0e-23
inj = {
    "F0": 100.0,
    "F1": -1e-9,
    "F2": 0.0,
    "Alpha": 0.0,
    "Delta": 0.0,
    "h0": sqrtSX / depth,
    "cosi": 1,
    "psi": 0.0,
    "phi": 0.0,
    "tref": tstart
}

In [ ]:
# Loading data
logger.info("Loading SFT data and computing normalized power...")
freqs, times, sft_data = pyfstat.utils.get_sft_as_arrays(sftfilepath)
sft_power = sft_data["H1"].real ** 2 + sft_data["H1"].imag ** 2
normalized_power = (
    2 * sft_power / (Tsft * sqrtSX ** 2)
)

# Plot spectrogram with plot_triplets function

Here, we can make out two modulations. One is due to the instrinsic spin-down of the source, and the other is due to the relative motion of the earth around the sun w.r.t. the source.

In equations, the frequency evolution of the signal can be described as:

\begin{equation}
f(t) = \left(f_0+\dot{f}(t-t_0)\right)\left(1+\frac{\vec{v}\cdot\hat{n}}{c}\right) \tag{1}
\end{equation}
where $f_0$ is the emitted GW frequency at the time $t_0$, $\vec{v}$ is the velocity of the detector, $\hat{n}$ is the unit vector between the earth and the source.

$f(t)$ is the measured frequency at time $t$, and is what is plotted in the time/freuqency maps below.

We are interested in obtaining the source parameters, that is, the GW frequency $f_0$ and the spin-down $\dot{f}$.



In [ ]:
flat_times,flat_freqs,flat_pows = pyhough.pm.flatten_spectrogram(times["H1"],freqs,normalized_power)
pyhough.pm.python_plot_triplets((flat_times-flat_times[0])/86400,flat_freqs,flat_pows,'.',label='equalized power')
plt.xlabel('time (days)',size=14)
plt.ylabel('frequency (Hz)',size=14);

# Create peakmap from spectrogram

Here, we take the spectrogram computed in PyFstat, and, in each Fast Fourier Transform (that is, at each time in the spectrogram), select local maxima above a chosen threshold. The local maxima criteria reduce the impact of noise lines that may pollute multiple adjacent frequency bins.

We call the resulting thresholded spectrogram a "peakmap", where each surviving time/frequency pixel is called a "peak".

#### Setting threshold on peak selection to be:

In [ ]:
threshold = 3 # need to apply a threshold on equalized power and select maxima above this threshold to run Hough
pm_times,pm_freqs,pm_pows,index = pyhough.pm.make_peakmap_from_spectrogram(times["H1"],freqs,normalized_power,threshold)

In [ ]:
pyhough.pm.python_plot_triplets((pm_times-pm_times[0])/86400,pm_freqs,pm_pows,'.',label='equalized power')
plt.xlabel('time (days)',size=14)
plt.ylabel('frequency (Hz)',size=14);
# plt.ylim([99.98,100.02])

# Create source position vector and get detector velocities for Dopp Corr

In [ ]:
# source position and detector velocities
writer_kwargs = {
    "label": "SingleDetectorGaussianNoiseSignalInjected",
    "outdir": "Mockdata",
    "tstart": 1238166018,
    "duration": 0.4 * 365 * 86400,
    "detectors": "H1",
    "sqrtSX": sqrtSX,
    "Tsft": 1800,
    "SFTWindowType": "tukey",
    "SFTWindowParam": 0.01,
}
writer = pyfstat.Writer(**writer_kwargs, **inj)
position = [inj["Alpha"], inj["Delta"]]
vec_n = pyhough.pm.astro2rect(position,1)
ts,vs = pyhough.pm.get_detector_velocities(writer)
Nts = len(ts)

# Remove Doppler shift from peakmap

To search for a continuous wave coming from a particular sky location, we "point" our detector by correcting the data for the Doppler shift induced by the relative motion of earth and the source. In this method, the Doppler correction is a shifting of the time/frequency peaks in the peakmap for a given sky location.

Essentially, this corresponds to the following

\begin{equation}
\frac{f(t)}{\left(1+\frac{\vec{v}\cdot\hat{n}}{c}\right)} = f_0 +\dot{f}(t-t_0) \tag{2}
\end{equation}

Now, the new frequency is simply:

\begin{equation}
f_{\rm new}(t) = f_0 + \dot{f}(t-t_0)
\end{equation}

and we can apply the frequency-Hough Transform to search for this modulation.




In [ ]:
pm_freqs_undop = pyhough.pm.remove_doppler_from_peakmap(pm_times,pm_freqs,index,vec_n,vs,Nts) # Eq. 2
pyhough.pm.python_plot_triplets((pm_times-pm_times[0])/86400,np.array(pm_freqs_undop),pm_pows,'.',label='equalized power')
# plt.ylim([99.989,100.002]);
plt.xlabel('time (days)',size=14)
plt.ylabel('frequency (Hz)',size=14);

# Analysis and relevant signal parameters

In [ ]:
alltimes = times["H1"] ### the times of the analysis
df = 1/Tsft # the frequency bin size
ref_perc_time = 0.0 * 100 # reference time for the Hough at which f0 is determined, set any number between 0 (beginning), 100 (end)
sig_fdot = inj['F1'] #spin-down of injected signal
dsd = 1/(Tsft * duration) # step in spin-down: dsd = df / Tobs
sdgrid = pyhough.hm.make_sd_grid(sig_fdot,dsd) ## grid of spin-downs to search over

# Run frequency-Hough on Doppler corrected peakmap and plot Hough map


The input is the time/frequency peakmap, which is simply a collection of ones. For given choices of $\dot{f}$, we solve for $f_0$

\begin{equation}
f_0 = f_{\rm new}-\dot{f}(t-t_0)
\end{equation}

We thus need to specify a range of $\dot{f}$ values with a particular spacing, given by:

\begin{equation}
\delta\dot{f} = df/T_{\rm obs}
\end{equation}
where $\delta f=1/T_{\rm FFT}$ and $T_{\rm FFT}$ is the FFT length (1800 s in this case), and specify a maximum and minimum $\dot{f}$ to search over.

The Hough sums ones along the time/frequency tracks of the signals, NOT the equalized power on the color axis that is seen in the peakmap plots

The output of this code $\texttt{hmap_from_pm}$ is a 2-dimensional histogram in the $f_0$/$\dot{f}$ plane of the source.

In [ ]:
hmap_from_pm = pyhough.hm.hfdf_hough(pm_times,pm_freqs_undop,Tsft,sdgrid,ref_perc_time)
fs_for_hmap_from_pm = np.arange(np.min(pm_freqs_undop),np.max(pm_freqs_undop),df)

In [ ]:
fig, ax = plt.subplots()#figsize=(0.8 * 16, 0.8 * 9))
ax.set(ylabel="Spin-down [Hz/s]", xlabel=r"Frequency [Hz]")#, ylim=(99.98, 100.02))
c = ax.pcolormesh(
    fs_for_hmap_from_pm,
    sdgrid,
    hmap_from_pm,
    cmap="inferno",
    shading="nearest",
)
fig.colorbar(c, label="number count")
plt.tight_layout()
# ax.set(ylim=(sig_fdot*1.3,-0.7e-9));
ax.set(xlim=(99.975,100.025));
plt.savefig(os.path.join(outdir, label + "_hm_map.png"), format='png',dpi=400)

In [ ]:
# Zoom in at peak
fig, ax = plt.subplots()#figsize=(0.8 * 16, 0.8 * 9))
ax.set(ylabel="Spin-down [Hz/s]", xlabel=r"Frequency [Hz]")#, ylim=(99.98, 100.02))
c = ax.pcolormesh(
    fs_for_hmap_from_pm,
    sdgrid,
    hmap_from_pm,
    cmap="inferno",
    shading="nearest",
)
fig.colorbar(c, label="number count")
plt.tight_layout()
ax.set(ylim=(inj["F1"] - 20*dsd, inj["F1"] + 20*dsd));
ax.set(xlim=(99.975,100.025));
plt.savefig(os.path.join(outdir, label + "_hm_map_zoom.png"), format='png',dpi=400)

Q1: Why are there seemingly negative slopes of the lines in this map? Can you guess which parameter you need to change to obtain all positive slopes or a mix of positive and negative sloped lines?

Q2: If the true signal frequency and spin-down are  F0 and F1, respectively, why are there multiple pixels in this map surrounding the true values?

# Apply wrong Doppler correction to peakmap, run Hough, and plot

In [ ]:
wrong_position = [np.pi/3, np.pi/4]
wrong_vec_n = pyhough.pm.astro2rect(wrong_position,1)

In [ ]:
pm_freqs_undop_wrong = pyhough.pm.remove_doppler_from_peakmap(pm_times,pm_freqs,index,wrong_vec_n,vs,Nts)
pyhough.pm.python_plot_triplets((pm_times-pm_times[0])/86400,np.array(pm_freqs_undop_wrong),pm_pows,'.',label='equalized power')
# plt.ylim([99.989,100.002]);
plt.xlabel('time (days)',size=14)
plt.ylabel('frequency (Hz)',size=14);

Q3: Do you see the difference between using the wrong and correct source positions to correct the peakmap? What can you conclude about the ability of this method to localize sources in the sky?

Q4: What do you think will happen to a monochromatic noise line after the Doppler correction is applied? Is this good or not? Why?


Comment: Noise line is one of the non-Gaussian noise. It can be modeled by a sinusoidal waveform in the detector frame:

$$ h_\mathrm{line}(t) = A \sin(2\pi f_\mathrm{line} t + \phi_\mathrm{line}) $$


In [ ]:
# ### run Hough with wrong position
hmap_wrong_pos = pyhough.hm.hfdf_hough(pm_times,pm_freqs_undop_wrong,Tsft,sdgrid,ref_perc_time)
fs_for_hmap_from_spec_wrong_position = np.arange(np.min(pm_freqs_undop_wrong),np.max(pm_freqs_undop_wrong),df)

In [ ]:
fig, ax = plt.subplots()#figsize=(0.8 * 16, 0.8 * 9))
ax.set(ylabel="Spin-down [Hz/s]", xlabel=r"Frequency at $t_0$ [Hz]")#, ylim=(99.98, 100.02))
c = ax.pcolormesh(
    fs_for_hmap_from_spec_wrong_position,
    sdgrid,
    hmap_wrong_pos,
    cmap="inferno",
    shading="nearest",
)
fig.colorbar(c, label="number count")
plt.tight_layout()
plt.savefig(os.path.join(outdir, label + "_hm_map_wrongposition.png"), format='png',dpi=400)
# ax.set(ylim=(sig_fdot*2,-sig_fdot*1.1));

Q5: Compare the peak number count at the source frequency and spin-down in this Hough map and the one run on the properly corrected peakmap. Why is there a reduction in the number count?



In [ ]:
%reset